In [9]:
import requests
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

###### Input your access_token here!######
# access_token = 'xxxxxxxxxxxxxxxxxxxxxxxx'

infile = 'https://raw.githubusercontent.com/alvin-chan/bdt_django_front_end_postgres_mkdocs/master/example_data/bdt_example_input.csv'

input_df = pd.read_csv(infile)

# Plotting the historical demand
fig = px.bar(input_df[:100], x='timestamp', y=['demand_units'], title='Training Data')
fig = px.bar(input_df[100:130], x='timestamp', y=['demand_units'], title='Test Demand Data')

fig = go.Figure()
fig.update_layout(title="Training & Test Demand Data")
fig.add_trace(
    go.Bar(x=input_df['timestamp'].values[:100], y=input_df['demand_units'].values[:100],
           name="Training Data"))
fig.add_trace(
    go.Bar(x=input_df['timestamp'][100:130].values, y=input_df['demand_units'].values[100:130],
          name="Test Data"))
fig.show()

In [10]:
# Here, we're defining the scenario:

# As a restaurant, I want to know how much to prepare the next 2 days for consumer demand, 
# The item is number of burger patties (1 patty per burger), my patties have a shelf life of 2 days
# Deliveries occur every day of the week
# Each burger patty costs $1, and is sold for $8
# Simulate what would happen if I forecast every 2 days, over a period of 30 days
# Use the first 100 days of the historical data as the training period (c. 3 months worth)
# Without WasteNot, 20% of the delivered stock is wasted


input_dict = {'timestamp':list(input_df['timestamp'].values),
              'demand':[int(x) for x in input_df['demand_units'].values],
              "cost": 1.0,
              "sale_price": 8.0, 
              "shelf_life_seconds": 172800.0, 
              "opt_param": "profit",
              "forecast_periods_ahead":2,
              "delivery_dayofweek": [0,1,2,3,4,5,6],
              "n_training_periods": 100,
              "n_forecasts_simulated": 15
              }

payload = json.dumps(input_dict)

# Here, we're taking the scenario and calling out backcast function

url = "https://api.bluedotthinking.com/backcast"

headers = {'Content-Type': 'application/json',
           'access_token': access_token}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)
json_data = json.loads(response.text)
print (response)

# Loading the time-series data from the JSON response into a dataframe, for easier analysis

simulation_df = pd.DataFrame(data={'timestamp':json_data['timestamp_start'],
                                   'actual_demand_units':json_data['actual_demand_units_unconstrained'],
                                   'simulation_satisfied_units':json_data['simulation_satisfied_units'],
                                   'simulation_delivered_units':json_data['simulation_delivered_units'],
                                   'simulation_unmet_units':json_data['simulation_unmet_units'],
                                   'simulation_wasted_units':json_data['simulation_wasted_units']
                                  })

/Users/alvin/.pyenv/versions/bdt-backend-env/lib/python3.8/site-packages/urllib3/connectionpool.py:979: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'api.bluedotthinking.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



<Response [200]>


In [11]:
## Here, we are showing the benefits deploying the system would have had:
from datetime import datetime

print ('Deploying the WasteNot System on this item between {0} & {1} would have had the following impact:'.format(json_data['simulation_first_datetime'], json_data['simulation_last_datetime']))

a = datetime.strptime(json_data['simulation_first_datetime'],"%Y-%m-%d")
b = datetime.strptime(json_data['simulation_last_datetime'],"%Y-%m-%d")
seconds_between = (b-a).total_seconds()
days_between = (b-a).days
seconds_in_week = 24*3600*7
seconds_in_year = 24*3600*365
months_in_year = 12
seconds_in_month = seconds_in_year/months_in_year

n_weeks_in_simulation = seconds_between/seconds_in_week
n_months_in_simulation = seconds_between/seconds_in_month

weekly_benefit = json_data['profit_change'] / n_weeks_in_simulation
monthly_benefit = json_data['profit_change'] / n_months_in_simulation


hist_kpi = [json_data['historical_profit'], json_data['historical_revenue'], json_data['historical_cost'], json_data['historical_demand_units_total'], json_data['historical_wasted_units_total']]
simulation_kpi = [json_data['simulation_profit'], json_data['simulation_revenue'], json_data['simulation_cost'], json_data['simulation_satisfied_units_total'], json_data['simulation_wasted_units_total']]
abs_change_in_kpi = [simulation_kpi[x] - hist_kpi[x] for x in range(len(simulation_kpi))]
percentage_change_in_kpi = [100.*(simulation_kpi[x] - hist_kpi[x])/hist_kpi[x] for x in range(len(simulation_kpi))]
percentage_change_in_kpi = [ '%.1f' % elem for elem in percentage_change_in_kpi ]

fig = go.Figure()
fig.add_trace(go.Indicator(
    mode = "number",
    value = json_data['simulation_profit']-json_data['historical_profit'],    
    title = {"text": "Net Business Benefit (Change in Profit)<br><span style='font-size:0.8em;color:gray'>"+str(days_between)+" days between "+json_data['simulation_first_datetime']+" & "+json_data['simulation_last_datetime']+"</span>"},    
    number = {'prefix': "$",
              'font.color': 'green'},
    domain = {'x': [0.0,0.475]
              , 'y': [0.5, 1.0]
             }
))

fig.add_trace(go.Indicator(
    mode = "delta",
    value = json_data['simulation_wasted_units_total'],
    title = {"text": "Wastage Reduction <br><span style='font-size:0.8em;color:gray'>"+str(days_between)+" days between "+json_data['simulation_first_datetime']+" & "+json_data['simulation_last_datetime']+"</span>"},
    delta = {'reference': json_data['historical_wasted_units_total'], 'relative': True, 'position' : "right",
             'increasing.color':"red",
             'decreasing.color':"green",},
    domain = {'x': [0.525,1.0]
              , 'y': [0.5, 1.0]
             }
))



fig.add_trace(go.Indicator(
    mode = "number",
    value = weekly_benefit,
    title = {"text": "Weekly Benefit"},    
    number = {'prefix': "$",
              'font.color': 'green'},
    domain = {'x': [0.0,0.475]
              , 'y': [0.0, 0.5]
             }
))

fig.add_trace(go.Indicator(
    mode = "number",
    value = monthly_benefit,
    title = {"text": "Monthly Benefit"}, 
    number = {'prefix': "$",
              'font.color': 'green'},
    domain = {'x': [0.525,1.0]
              , 'y': [0.0, 0.5]
             }
))

fig.show()

print ('Some Detailed KPIs on the simulation:')
fig2 = go.Figure(data=[go.Table(header=dict(values=['KPI','Without WasteNot', 'With WasteNot', 'Change in KPI', '% Change in KPI']),
                 cells=dict(values=[['Profit','Revenue','Cost','Satisfied Units','Wasted Units'],
                                    [json_data['historical_profit'], json_data['historical_revenue'], json_data['historical_cost'], json_data['historical_demand_units_total'], json_data['historical_wasted_units_total']],
                                    [json_data['simulation_profit'], json_data['simulation_revenue'], json_data['simulation_cost'], json_data['simulation_satisfied_units_total'], json_data['simulation_wasted_units_total']],
                                    abs_change_in_kpi,
                                    percentage_change_in_kpi,
                                   ]))
                     ])
fig2.show()

Deploying the WasteNot System on this item between 2017-09-01 & 2017-09-30 would have had the following impact:


Some Detailed KPIs on the simulation:


In [12]:
fig = go.Figure()
fig.update_layout(title="Actual Demand vs. Demand Satisfied using WasteNot")
fig.add_trace(
    go.Line(x=simulation_df['timestamp'].values, y=simulation_df['simulation_satisfied_units'].values,
           name="Demand Satisfied using WasteNot", line_color='blue'))
fig.add_trace(
    go.Line(x=simulation_df['timestamp'].values, y=simulation_df['actual_demand_units'].values,
          name="Actual Demand Units", line_color='black'))
fig.show()


fig2 = go.Figure()
fig2.update_layout(title="Wasted Units & Unmet Demand, Using WasteNot")
fig2.add_trace(
    go.Bar(x=simulation_df['timestamp'].values, y=simulation_df['simulation_wasted_units'].values,
           name="Wasted Units using WasteNot"))
fig2.add_trace(
    go.Bar(x=simulation_df['timestamp'].values, y=simulation_df['simulation_unmet_units'].values,
          name="Unmet Demand Units"))
fig2.show()

fig3 = go.Figure()
fig3.update_layout(title="Actual Demand vs. Delivered Units using WasteNot")
fig3.add_trace(
    go.Bar(x=simulation_df['timestamp'].values, y=simulation_df['simulation_delivered_units'].values,
           name="Delivered Units using WasteNot"))

fig3.add_trace(
    go.Line(x=simulation_df['timestamp'].values, y=simulation_df['actual_demand_units'].values,
          name="Actual Demand Units", line_color='black'))
fig3.show()